In [1]:
import warnings
warnings.simplefilter(action='ignore', category=FutureWarning)

In [2]:
try:
    t = t
except NameError:
    clean_up = True
    %run StdPackages.ipynb
    name, t = 'uniform', 1990
    ws = gams.GamsWorkspace(working_directory=d['work']) # specify where you want to run the GAMS models from (here the repository referred to in d['work'])
    ns = [22, 30, 'T']
    # what type of settings do we use to solve the various models:
    kwargs_SY = {'name': 'SY', 'loop': 'l_SY', 'n': 10}
    kwargs_IIa= {'name': 'IIa', 'loop': 'l_IIa', 'n': 10}
    kwargs_IIb= {'name': 'IIb', 'loop': 'l_IIb', 'n': 3}
    kwargs_IIIa = {ns[0]: {'name': f'IIIa_{ns[0]}', 'loop': f'l_IIIa_{ns[0]}', 'n': 10}}
    [kwargs_IIIa.__setitem__(n, {'name': f'IIIa_{n}', 'loop': f'l_IIIa_{n}', 'n': 3}) for n in ns[1:]]
    kwargs_IIIb = {ns[0]: {'name': f'IIIb_{ns[0]}', 'loop': f'l_IIIb_{ns[0]}', 'n': 10}}
    [kwargs_IIIb.__setitem__(n, {'name': f'IIIb_{n}', 'loop': f'l_IIIb_{n}', 'n': 3}) for n in ns[1:]]
    kwargs_IIIc = {ns[0]: {'name': f'IIIc_{ns[0]}', 'loop': f'l_IIIc_{ns[0]}', 'n': 10}}
    [kwargs_IIIc.__setitem__(n, {'name': f'IIIc_{n}', 'loop': f'l_IIIc_{n}', 'n': 3}) for n in ns[1:]]
    
with open(f"{d['processedData']}\\glob_{t}","rb") as file: # load global settings anmed glob_name
    glob=pickle.load(file)
db_IO = GpyDB(pickle_path = os.path.join(d['processedData'], f'{name}_{t}')) # load IO database named IO_name
addTimeToDb(db_IO, glob.db['t'].vals, exceptions = ['sigma'])
with open(f"{d['modules']}\\{name}_GE_{t}","rb") as file:
    GE = pickle.load(file)
GE.s.db['softConstr'] = 0.005

Proceed by:
1. Sneak up on the solution to the uniform CO2 tax.
2. Solve for the "single year target" (SY) approach.
3. Solve for the case with "continued policy", but without elimination of overshoot.
4. Solve for the case with elimination in the entire transitory phase.

In [3]:
GE.s.db['qCO2Target'] = adj.rc_pd(GE.get('qCO2agg'), db_IO.get('MTarget_SY'))
mEmissions = GE.m[f'{name}_{t}_emissions']
mEmissions.readTargetSubsets()

*1. Move ```tauDist``` to equal weights (uniform CO2 tax):*

In [4]:
dbUniform = GpyDB(**{'name': 'taudist'})
dbUniform['tauDist'] = pd.Series(1, index = GE.get('tauDist')[GE.get('tauDist')!=0].index, name = 'tauDist')
m_uniform, cp_uniform = GE.sneakySolve(dbUniform, ws = ws, loop_kwargs = {'n': 10}, **{'cns': 'CONOPT4'})

*2. Solve for the singly-year approach:*

In [5]:
GE.s.setstate('Mbinding')
GE.s.states['Mbinding']['text'] = {'Run': GE.s.writeSolveState('Mbinding')}
dbSY = GpyDB(**{'name': 'SY'})
dbSY['qCO2Target'] = db_IO.get('MTarget_SY')
m_sy, cp_sy = GE.sneakySolve(dbSY, ws = ws, options_add = {'checkpoint': cp_uniform}, loop_kwargs = kwargs_SY, **{'cns':'CONOPT4'})

*3. Use the solution from the single-year scenario and restart with the new targets:*
* Update solution
* Start the target at the solution path from earlier.
* Update relevant dummies (what subset of years include targets now).
* Rewrite model-code to start execution over.
* Create "target database" with the real targets that we gradually get close to. 

*Drop the current year's target?*

In [13]:
db_IO['MTarget_2A'] = db_IO.get('MTarget_2A').loc[t+1:]
db_IO['MTarget_2B'] = db_IO.get('MTarget_2B').loc[t+1:]

*Update data and subsets - define target database:*

In [14]:
robust.robust_merge_dbs(GE.s.db, m_sy.out_db.getTypes(['variable','scalar_variable']), priority = 'second') # update solution
GE.s.db['qCO2Target'] = adj.rc_pd(GE.get('qCO2agg'), db_IO.get('MTarget_2A')) # start targets from solution
mEmissions.readTargetSubsets() # update the relevant dummies
GE.write();
db_IIa = GpyDB(**{'name': 'IIa'})
db_IIa['qCO2Target'] = db_IO.get('MTarget_2A')

*Define function that tries to solve with binding constraints, and then relax the constraint if the shadow values are negative:*

In [15]:
def solveTrySoft(dbTarget, loop_kwargs, checkpoint = None, checkpointSoft = None, checkpointSoftText = None):
    checkpoint = {'checkpoint': checkpoint} if checkpoint else None
    m, cp = GE.sneakySolve(dbTarget, ws = ws, options_add = checkpoint, loop_kwargs = loop_kwargs, **{'cns':'CONOPT4'})
    if m.out_db.get('tauCO2agg').xs(t)<0:
        GE.s.setstate('MsoftConstr')
        if checkpointSoft:
            GE.s['text'] = {'Run': f"""solve {GE.s['name']} using CNS;"""} if checkpointSoftText is None else {'Run': checkpointSoftText}
            m, cp_ = GE.sneakySolve(dbTarget, ws = ws, options_add = {'checkpoint': checkpointSoft}, 
                                    loop_kwargs = {'name': loop_kwargs['name']+'_soft', 'n': 10, 'loop': loop_kwargs['loop']+'_soft'}, **{'cns':'CONOPT4'})
        else:
            GE.write();
            m, cp_ = GE.sneakySolve(dbTarget, ws = ws, loop_kwargs = {'name': loop_kwargs['name']+'_soft', 'n': 10, 'loop': loop_kwargs['loop']+'_soft'}, **{'cns':'CONOPT4'})
    else:
        cp_ = None
    return m, cp, cp_

*For IIa:*

In [16]:
m_IIa, cp_IIa, cp_IIa_soft = solveTrySoft(db_IIa, kwargs_IIa)

*For IIb: Start solution from checkpoint IIa*

In [17]:
GE.s.setstate('Mbinding') # reset state to Mbinding
db_IIb = GpyDB(**{'name': 'IIb'})
db_IIb['qCO2Target'] = db_IO.get('MTarget_2B')
GE.s['text'] = {'Run': f"""solve {GE.s['name']} using CNS;"""}
m_IIb, cp_IIb, cp_IIb_soft = solveTrySoft(db_IIb, kwargs_IIb, checkpoint = cp_IIa)

*Type III - target databases:*

In [18]:
def IIItarget(x, n):
    db = GpyDB(**{'name': f'III{x}_{n}'})
    db['qCO2Target'] = db_IO.get(f'MTarget_3{x}_n{n}')
    return db
dbs_IIIa = {n: IIItarget('A', n) for n in ns}
dbs_IIIb = {n: IIItarget('B', n) for n in ns}
dbs_IIIc = {n: IIItarget('C', n) for n in ns}

*Run IIIa types and store:*

In [19]:
checkpointSoftText = f"tTarget['{t}'] = no;\n"+GE.s.writeSolveState('MsoftConstr')
GE.s.setstate('Mbinding') # reset state to Mbinding
GE.s['text'] = {'Run': f"tTarget['{t}'] = no;\n"+GE.s['text']['Run']}
ms_IIIa = {n: None for n in ns}
cps_IIIa = {n: None for n in ns}
cps_IIIa_soft = {n: None for n in ns}
ms_IIIa[ns[0]], cps_IIIa[ns[0]], cps_IIIa_soft[ns[0]] = solveTrySoft(dbs_IIIa[ns[0]], 
                                                                     kwargs_IIIa[ns[0]], 
                                                                     checkpoint = cp_IIa, 
                                                                     checkpointSoft = cp_IIa, 
                                                                     checkpointSoftText = checkpointSoftText)
for n in ns[1:]:
    GE.s.setstate('Mbinding') # reset state to Mbinding
    ms_IIIa[n], cps_IIIa[n], cps_IIIa_soft[n] = solveTrySoft(dbs_IIIa[n], 
                                                             kwargs_IIIa[n], 
                                                             checkpoint = cps_IIIa[ns[ns.index(n)-1]],
                                                             checkpointSoft = cp_IIa, 
                                                             checkpointSoftText = checkpointSoftText)

*Run IIIb types and store:*

In [12]:
GE.s.setstate('Mbinding') # reset state to Mbinding
ms_IIIb = {n: None for n in ns}
cps_IIIb = {n: None for n in ns}
cps_IIIb_soft = {n: None for n in ns}
ms_IIIb[ns[0]], cps_IIIb[ns[0]], cps_IIIb_soft[ns[0]] = solveTrySoft(dbs_IIIb[ns[0]], 
                                                                     kwargs_IIIb[ns[0]], 
                                                                     checkpoint = cp_IIb, 
                                                                     checkpointSoft = cp_IIb, 
                                                                     checkpointSoftText = checkpointSoftText)
for n in ns[1:]:
    GE.s.setstate('Mbinding') # reset state to Mbinding
    ms_IIIb[n], cps_IIIb[n], cps_IIIb_soft[n] = solveTrySoft(dbs_IIIb[n], 
                                                             kwargs_IIIb[n], 
                                                             checkpoint = cps_IIIb[ns[ns.index(n)-1]],
                                                             checkpointSoft = cp_IIb, 
                                                             checkpointSoftText = checkpointSoftText)

*Run IIIc types and store:*

In [13]:
GE.s.setstate('Mbinding') # reset state to Mbinding
ms_IIIc = {n: None for n in ns}
cps_IIIc = {n: None for n in ns}
cps_IIIc_soft = {n: None for n in ns}
ms_IIIc[ns[0]], cps_IIIc[ns[0]], cps_IIIc_soft[ns[0]] = solveTrySoft(dbs_IIIc[ns[0]], 
                                                                     kwargs_IIIc[ns[0]], 
                                                                     checkpoint = cp_IIb, 
                                                                     checkpointSoft = cp_IIb, 
                                                                     checkpointSoftText = checkpointSoftText)
for n in ns[1:]:
    GE.s.setstate('Mbinding') # reset state to Mbinding
    ms_IIIc[n], cps_IIIc[n], cps_IIIc_soft[n] = solveTrySoft(dbs_IIIc[n], 
                                                             kwargs_IIIc[n], 
                                                             checkpoint = cps_IIIc[ns[ns.index(n)-1]],
                                                             checkpointSoft = cp_IIb, 
                                                             checkpointSoftText = checkpointSoftText)

Export:

In [14]:
exportTo = os.path.join(d['curr'], 'climateBudgets')
m_uniform.out_db.export(repo = exportTo, name = f'{name}_{t}_uniform')
m_sy.out_db.export(repo = exportTo, name = f'{name}_{t}_SY')
m_IIa.out_db.export(repo = exportTo, name = f'{name}_{t}_IIa') 
m_IIb.out_db.export(repo = exportTo, name = f'{name}_{t}_IIb') 
[m.out_db.export(repo = exportTo, name = f'{name}_{t}_IIIa{n}') for n,m in ms_IIIa.items()];
[m.out_db.export(repo = exportTo, name = f'{name}_{t}_IIIb{n}') for n,m in ms_IIIb.items()];
[m.out_db.export(repo = exportTo, name = f'{name}_{t}_IIIc{n}') for n,m in ms_IIIc.items()];